# Mapping broadcasted news related to protests

In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [ ]:
from arcgis import GIS
from arcgis.features import FeatureSet
from datetime import datetime, timedelta
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat
from georapid.protests import aggregate, articles, hotspots

## Create a client instance

In [ ]:
host = "geoprotests.p.rapidapi.com"
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Connect to ArcGIS Online anonymously

In [ ]:
gis = GIS()

## Define some utility functions
We need to plot hotspot locations and aggregated polygons.

In [ ]:
def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        spatial_df.spatial.plot(map_view,
                                renderer_type='c', # for class breaks renderer
                                method='esriClassifyNaturalBreaks', # classification algorithm
                                class_count=5, # choose the number of classes
                                col=column, # numeric column to classify
                                cmap='YlOrRd', # color map to pick colors from for each class
                                alpha=0.35 # specify opacity
        )
        
def plot_hotspots(map_view, spatial_df):
    """
    Plots the spatial dataframe as red locations using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        spatial_df.spatial.plot(map_view,
                                renderer_type='s', # for simple renderer
                                symbol_style='o', # symbol style circle
                                colors='Reds_r', # colors red
                                alpha=0.35 # specify opacity
        )

### Visualize the news related to protests from yesterday

In [ ]:
world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

protests_featureset = FeatureSet.from_dict(aggregate(client, format=OutFormat.ESRI))
plot_aggregated(world_map, protests_featureset.sdf)
world_map

### Visualize the news hotspots related to protests from yesterday
The map stays empty if no specific location were mentioned many times in news related to protests, yesterday.

In [ ]:
world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

hotspots_featureset = FeatureSet.from_dict(hotspots(client, format=OutFormat.ESRI))
plot_hotspots(world_map, hotspots_featureset.sdf)
world_map

### Visualize the news related to protests of 24th February 2022

In [ ]:
date_of_interest = datetime(2022, 2, 24)

world_map = gis.map()
world_map.basemap = 'dark-gray-vector'

protests_featureset = FeatureSet.from_dict(aggregate(client, date_of_interest, OutFormat.ESRI))
plot_aggregated(world_map, protests_featureset.sdf)
world_map